In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [5]:
#TRAIN_DATA_PATH = "/content/drive/MyDrive/機械学習コンペ/dataset/train_Mod.csv"
TRAIN_DATA_PATH = "dataset/train_Mod.csv"
train_data = pd.read_csv(TRAIN_DATA_PATH)

train_data

,id,knee_depth,hip_depth,misalignment,width_of_hand,jump_height,personal_id
0,0,4.114675,20.081082,13.244238,-14.312221,44.388231,A
1,1,2.194313,16.056999,3.360566,-13.784517,45.885170,A
2,2,6.526472,15.633804,3.355818,-0.642056,41.962244,A
3,3,2.307960,17.281763,13.097795,-6.454952,45.614645,A
4,4,3.076618,16.892115,10.723250,-6.547873,44.228786,A
...,...,...,...,...,...,...,...
607,607,10.834146,17.801688,-9.491518,-6.994127,27.669528,C
608,608,14.595132,15.752806,16.056642,5.227412,27.967953,C
609,609,7.543161,15.813522,-5.888387,1.361989,28.125805,C
610,610,7.423551,13.360349,-2.535648,-3.225412,21.747075,C


In [6]:
# 外れ値の除去
df_ = train_data[train_data['knee_depth'] < 17.5]
df_ = df_[df_['jump_height'] < 80]
df_dropped = df_.dropna()
train_data, valid_data = train_test_split(df_dropped, test_size=0.2, random_state=0)
X_train = train_data.drop("personal_id", axis=1)
y_train = train_data["personal_id"]
X_valid = valid_data.drop("personal_id", axis=1)

y_valid = valid_data["personal_id"]


In [7]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

#最適なパラメータで保存
param = {
    'classifier__colsample_bytree': [1.0], 
    'classifier__gamma': [0.2], 
    'classifier__learning_rate': [0.3], 
    'classifier__max_depth': [5], 
    'classifier__min_child_weight': [1], 
    'classifier__n_estimators': [50], 
    'classifier__subsample': [0.9]}
pipeline = Pipeline([
    ('classifier', XGBClassifier(eval_metric='mlogloss'))  # ステップ3: XGBoost
])

#yのラベリング
train_y = train_data["personal_id"]
valid_y = valid_data["personal_id"]
# LabelEncoderのインスタンスを作成
label_encoder = LabelEncoder()
# クラスラベルを数値に変換
y_train_encoded = label_encoder.fit_transform(train_y)
y_valid_encoded = label_encoder.fit_transform(valid_y)


model = GridSearchCV(pipeline, param, cv=5, scoring='accuracy', n_jobs=-1)
model.fit(X_train, y_train_encoded)

#最適なパラメータを表示
print("the best param is : ", model.best_params_)
print("the best score is : ", model.best_score_)
#検証データで評価
y_pred = model.predict(X_valid)
print("valid accuracy_score : ", accuracy_score(y_valid_encoded, y_pred))

best_xgb = model.best_estimator_




the best param is :  {'classifier__colsample_bytree': 1.0, 'classifier__gamma': 0.2, 'classifier__learning_rate': 0.3, 'classifier__max_depth': 5, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 50, 'classifier__subsample': 0.9}
the best score is :  0.9710910652920962
valid accuracy_score :  0.9669421487603306


f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

In [11]:
#テストデータで評価
test_data = pd.read_csv("./dataset/test.csv")
X_test = test_data.drop("personal_id", axis=1)

y_test_pred = best_xgb.predict(X_test)

#エンコードしたラベルを元に戻す
y_test_pred = label_encoder.inverse_transform(y_test_pred)
#提出用ファイルの作成
test_data.loc[:, 'personal_id'] = y_test_pred
#特徴量の列を削除してcsv化
columns_to_save = ['id', 'personal_id']
test_data.to_csv('submission5.csv', columns=columns_to_save, index=True)

f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

In [17]:
#テスト用
test_path = "./dataset/train_Mod_test.csv"
test_data = pd.read_csv(test_path)

# 外れ値の除去
df_ = train_data[train_data['knee_depth'] < 17.5]
df_ = df_[df_['jump_height'] < 80]
df_dropped = df_.dropna()

y_test = best_xgb.predict(df_dropped.drop("personal_id", axis=1))

#確認
y_test = label_encoder.inverse_transform(y_test)
print(y_test)
#元データを読み込み
origin_data = pd.read_csv("./dataset/train_Mod.csv")
# 外れ値の除去
df_ = origin_data[origin_data['knee_depth'] < 17.5]
df_ = df_[df_['jump_height'] < 80]
df_dropped = df_.dropna()

origin_data_y = df_dropped["personal_id"]
#ndarrayに変換
origin_data_y = origin_data_y.values
print(origin_data_y)

for predict_v, origin_v in zip(y_test, origin_data_y):
    if predict_v != origin_v:
        print("違うものがある", predict_v, origin_v)





['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'C'
 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C'
 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C'
 'C' 'C' 'C' 'C' 'C' 'C' 'A' 'A' 'A' 'A' 'A' 'A' 'A

f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

In [ ]:
#LightGBM
import lightgbm as lgb

# LightGBMのモデルを作成
lgbm = lgb.LGBMClassifier(objective='multiclass', num_class=6)

# GridSearchで探索するパラメータの設定
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7]
}

# GridSearchCVの設定
grid_search = GridSearchCV(lgbm, param_grid, cv=3)

# ハイパーパラメータ探索
grid_search.fit(X_train, y_train)

# 最適なパラメータとその時のスコアを表示
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

# 最適なパラメータでモデルを訓練
best_lgbm = grid_search.best_estimator_

# テストデータで評価
y_pred = best_lgbm.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print(f"Valid accuracy: {accuracy}")



In [2]:
#テストデータで評価
test_data = pd.read_csv("./dataset/test.csv")
X_test = test_data.drop("personal_id", axis=1)

y_test_pred = best_lgbm.predict(X_test)

#提出用ファイルの作成
test_data.loc[:, 'personal_id'] = y_test_pred
#特徴量の列を削除してcsv化
columns_to_save = ['id', 'personal_id']
test_data.to_csv('submission.csv', columns=columns_to_save, index=True)

NameError: name 'pd' is not defined

In [7]:
import lightgbm as lgb

param = {
    'learning_rate': [0.1], 'max_depth': [5], 'n_estimators': [50]
}

# LightGBMのモデルを作成
lgbm = lgb.LGBMClassifier(objective='multiclass', num_class=6)

# GridSearchCVの設定
grid_search = GridSearchCV(lgbm, param, cv=3)

# ハイパーパラメータ探索
grid_search.fit(X_train, y_train)

# 最適なパラメータとその時のスコアを表示
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

# 最適なパラメータでモデルを訓練
best_lgbm = grid_search.best_estimator_

# テストデータで評価
y_pred = best_lgbm.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print(f"Valid accuracy: {accuracy}")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000639 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 649
[LightGBM] [Info] Number of data points in the train set: 322, number of used features: 6
[LightGBM] [Info] Start training from score -1.749200
[LightGBM] [Info] Start training from score -1.749200
[LightGBM] [Info] Start training from score -1.842726
[LightGBM] [Info] Start training from score -1.470486
[LightGBM] [Info] Start training from score -2.060979
[LightGBM] [Info] Start training from score -1.990362
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [8]:
# アンサンブル学習の適用

from sklearn.ensemble import VotingClassifier

# アンサンブルモデルの設定（Voting Classifier）
ensemble_model = VotingClassifier(estimators=[('xgb', best_xgb), ('lgb', best_lgbm)], voting='soft')
# モデルの訓練
ensemble_model.fit(X_train, y_train)

# テストデータで評価
y_pred = ensemble_model.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print(f"Valid accuracy: {accuracy}")

f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 970
[LightGBM] [Info] Number of data points in the train set: 483, number of used features: 6
[LightGBM] [Info] Start training from score -1.737365
[LightGBM] [Info] Start training from score -1.737365
[LightGBM] [Info] Start training from score -1.849283
[LightGBM] [Info] Start training from score -1.479536
[LightGBM] [Info] Start training from score -2.069143
[LightGBM] [Info] Start training from score -1.990362
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

In [19]:
#テスト用
test_path = "./dataset/train_Mod_test.csv"
test_data = pd.read_csv(test_path)

# 外れ値の除去
df_ = train_data[train_data['knee_depth'] < 17.5]
df_ = df_[df_['jump_height'] < 80]
df_dropped = df_.dropna()

y_test = ensemble_model.predict(df_dropped.drop("personal_id", axis=1))

#元データを読み込み
origin_data = pd.read_csv("./dataset/train_Mod.csv")
# 外れ値の除去
df_ = origin_data[origin_data['knee_depth'] < 17.5]
df_ = df_[df_['jump_height'] < 80]
df_dropped = df_.dropna()

origin_data_y = df_dropped["personal_id"]
#ndarrayに変換
origin_data_y = origin_data_y.values
print(origin_data_y)

for predict_v, origin_v in zip(y_test, origin_data_y):
    if predict_v != origin_v:
        print("違うものがある", predict_v, origin_v)





[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'C'
 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C

f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead